In [ ]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
target_col = ['年度','売上月','部門','大分類','小分類','品目','品名','規格','販売数量','販売単価','販売金額']
uriagedf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_U_2019_2021.csv",usecols=target_col)
target_col = ['年度','仕入月','部門名','大分類CD','大分類名','品目','品名','規格','仕入数','仕入単価','仕入金額']
siiredf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_S_2007_2021.csv",usecols=target_col)
target_col = ['棚卸月','部門名','大分類CD','大分類名','品目','品名','規格','在庫数','在庫単価','在庫金額']
zaikodf_n = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/N_Z_2019_2021.csv",usecols=target_col)

u_s = pd.read_csv("/content/drive/MyDrive/Purchased_products/U-S.csv", encoding="cp932")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# uriagedf_n 整理
uriagedf_n = uriagedf_n.fillna(0)
uriagedf_n["売上月"] = uriagedf_n["売上月"].astype(int)
uriagedf_n["品目"] = uriagedf_n["品目"].astype(int)
uriagedf_n["販売数量"] = uriagedf_n["販売数量"].astype(int)
uriagedf_n["販売単価"] = uriagedf_n["販売単価"].astype(int)
uriagedf_n["販売金額"] = uriagedf_n["販売金額"].astype(int)
uriagedf_n["年度"] = uriagedf_n["年度"].astype(int)

uriagedf_n = uriagedf_n[uriagedf_n["年度"] == 2020]
uriagedf_n = uriagedf_n[uriagedf_n["部門"] == "15仕入商品部門"]

In [ ]:
# siiredf_n 整理
siiredf_n = siiredf_n.fillna(0)
siiredf_n["品目"] = siiredf_n["品目"].astype(int)
siiredf_n["仕入数"] = siiredf_n["仕入数"].astype(int)
siiredf_n["仕入単価"] = siiredf_n["仕入単価"].astype(int)
siiredf_n["仕入金額"] = siiredf_n["仕入金額"].astype(int)
siiredf_n["年度"] = siiredf_n["年度"].astype(int)

siiredf_n = siiredf_n[siiredf_n["年度"] == 2020]

In [ ]:
# 在庫データ分類
zaikodf_n = zaikodf_n[zaikodf_n["部門名"]=='15仕入商品部門']
kisyudf_n = zaikodf_n[zaikodf_n["棚卸月"]==202005]
kimatudf_n = zaikodf_n[zaikodf_n["棚卸月"]==202105]

In [ ]:
uriage_list = []
for i in range(len(uriagedf_n)):
    if uriagedf_n.iloc[i]['品目'] not in uriage_list:
        uriage_list.append(uriagedf_n.iloc[i]['品目'])

In [ ]:
# 製品品目<－仕入品目DICT
u_s = u_s.fillna(0)
u_s_dict = {}
for i in range(len(u_s)):
    if u_s.iloc[i]['品目.1'] != 0:
        u_s_dict[u_s.iloc[i]['品目.1'][:-2]] = u_s.iloc[i]['品目'][:-2]

In [ ]:
hinmoku_dict = {}
for i in range(len(uriagedf_n)):
    if (uriagedf_n.iloc[i]['大分類'], uriagedf_n.iloc[i]['品目']) in hinmoku_dict:
        hinmoku_dict[uriagedf_n.iloc[i]['大分類'], uriagedf_n.iloc[i]['品目']][0] += uriagedf_n.iloc[i]['販売金額']
    else:
        hinmoku_dict[uriagedf_n.iloc[i]['大分類'], uriagedf_n.iloc[i]['品目']] = [uriagedf_n.iloc[i]['販売金額'],0,0,0,0]

In [ ]:
daibunrui_dict = {}
for i in range(len(kisyudf_n)):
    if kisyudf_n.iloc[i]['大分類CD'] in daibunrui_dict:
        continue
    else:
        daibunrui_dict[kisyudf_n.iloc[i]['大分類CD']] = str(kisyudf_n.iloc[i]['大分類CD']) + "_" + kisyudf_n.iloc[i]['大分類名']
for i in range(len(kimatudf_n)):
    if kimatudf_n.iloc[i]['大分類CD'] in daibunrui_dict:
        continue
    else:
        daibunrui_dict[kimatudf_n.iloc[i]['大分類CD']] = str(kimatudf_n.iloc[i]['大分類CD']) + "_" + kimatudf_n.iloc[i]['大分類名']
for i in range(len(siiredf_n)):
    if siiredf_n.iloc[i]['大分類CD'] in daibunrui_dict:
        continue
    else:
        daibunrui_dict[siiredf_n.iloc[i]['大分類CD']] = str(siiredf_n.iloc[i]['大分類CD']) + "_" + siiredf_n.iloc[i]['大分類名']
daibunrui_dict

In [ ]:
for i in range(len(kisyudf_n)):
    if kisyudf_n.iloc[i]['品目'] in uriage_list:
        if (daibunrui_dict[kisyudf_n.iloc[i]['大分類CD']], kisyudf_n.iloc[i]['品目']) in hinmoku_dict:
            hinmoku_dict[(daibunrui_dict[kisyudf_n.iloc[i]['大分類CD']], kisyudf_n.iloc[i]['品目'])][1] += \
                                                                                        kisyudf_n.iloc[i]['在庫金額']
        else:
            hinmoku_dict[(daibunrui_dict[kisyudf_n.iloc[i]['大分類CD']], kisyudf_n.iloc[i]['品目'])] = \
                                                                                        [0,kisyudf_n.iloc[i]['在庫金額'],0,0,0]

In [ ]:
for i in range(len(kimatudf_n)):
    if kimatudf_n.iloc[i]['品目'] in uriage_list:
        if kimatudf_n.iloc[i]['大分類CD'] in daibunrui_dict:
            if (daibunrui_dict[kimatudf_n.iloc[i]['大分類CD']], kimatudf_n.iloc[i]['品目']) in hinmoku_dict:
                hinmoku_dict[(daibunrui_dict[kimatudf_n.iloc[i]['大分類CD']], kimatudf_n.iloc[i]['品目'])][3] += \
                                                                                        kimatudf_n.iloc[i]['在庫金額']
            else:
                hinmoku_dict[(daibunrui_dict[kimatudf_n.iloc[i]['大分類CD']], kimatudf_n.iloc[i]['品目'])] = \
                                                                                        [0,0,0,kimatudf_n.iloc[i]['在庫金額'],0]

In [ ]:
for i in range(len(siiredf_n)):
    if str(siiredf_n.iloc[i]["品目"]) in u_s_dict:
        if u_s_dict[str(siiredf_n.iloc[i]["品目"])] in uriage_list:
            if (daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], u_s_dict[str(siiredf_n.iloc[i]['品目'])]) in hinmoku_dict:
                hinmoku_dict[(daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], u_s_dict[str(siiredf_n.iloc[i]['品目'])])][2] += siiredf_n.iloc[i]['仕入金額']
            else:
                hinmoku_dict[(daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], u_s_dict[str(siiredf_n.iloc[i]['品目'])])] = [0,0,siiredf_n.iloc[i]['仕入金額'],0,0]
    else:
        if siiredf_n.iloc[i]["品目"] in uriage_list:
            if (daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], siiredf_n.iloc[i]['品目']) in hinmoku_dict:
                hinmoku_dict[(daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], siiredf_n.iloc[i]['品目'])][2] += siiredf_n.iloc[i]['仕入金額']
            else:
                hinmoku_dict[(daibunrui_dict[siiredf_n.iloc[i]['大分類CD']], siiredf_n.iloc[i]['品目'])] = [0,0,siiredf_n.iloc[i]['仕入金額'],0,0]

In [ ]:
hinmei_dict = {}
for i in range(len(uriagedf_n)):
    if uriagedf_n.iloc[i]['品目'] not in hinmei_dict:
        hinmei_dict[uriagedf_n.iloc[i]['品目']] = [uriagedf_n.iloc[i]['品名'], uriagedf_n.iloc[i]['規格']]
for i in range(len(kisyudf_n)):
    if kisyudf_n.iloc[i]['品目'] not in hinmei_dict:
        hinmei_dict[kisyudf_n.iloc[i]['品目']] = [kisyudf_n.iloc[i]['品名'], kisyudf_n.iloc[i]['規格']]
for i in range(len(kimatudf_n)):
    if kimatudf_n.iloc[i]['品目'] not in hinmei_dict:
        hinmei_dict[kimatudf_n.iloc[i]['品目']] = [kimatudf_n.iloc[i]['品名'], kimatudf_n.iloc[i]['規格']]
for i in range(len(siiredf_n)):
    if siiredf_n.iloc[i]["品目"] in u_s_dict:
        if u_s_dict[siiredf_n.iloc[i]["品目"]] not in hinmei_dict:
            hinmei_dict[u_s_dict[siiredf_n.iloc[i]["品目"]]] = [siiredf_n.iloc[i]['品名'], siiredf_n.iloc[i]['規格']]
    else:
        if siiredf_n.iloc[i]["品目"] not in hinmei_dict:
            hinmei_dict[siiredf_n.iloc[i]["品目"]] = [siiredf_n.iloc[i]['品名'], siiredf_n.iloc[i]['規格']]

In [ ]:
df = pd.DataFrame(list(hinmoku_dict.items()))
a_list = []
for i in range(len(df)):
    temp_list= []
    temp_list.append(df.loc[i, 0][0])
    temp_list.append(df.loc[i, 0][1])
    temp_list.append(hinmei_dict[df.loc[i, 0][1]][0])
    temp_list.append(hinmei_dict[df.loc[i, 0][1]][1])
    temp_list.append(df.loc[i, 1][0])
    temp_list.append(df.loc[i, 1][1])
    temp_list.append(df.loc[i, 1][2])
    temp_list.append(df.loc[i, 1][3])
    temp_list.append(df.loc[i, 1][4])
    a_list.append(temp_list)

In [ ]:
df = pd.DataFrame(a_list, columns=['大分類','品目','品名','規格','売上','期首','期中','期末','原価'])

In [ ]:
df.sum()

In [ ]:
df["原価"] = df["期首"] + df["期中"] - df["期末"]
df["粗利"] = df["売上"] - df["原価"]
df.head()

In [ ]:
# Excelデータ作成
df.to_excel("/content/drive/MyDrive/Purchased_products/仕入品資料Ⅱ.xlsx", sheet_name='月別粗利')